In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import uniform, randint
import joblib

# Load and clean data
filename = 'cov_new2.csv'
data = pd.read_csv(filename)

# Define the feature names
original_feature_names = ['MolecularWeight', 'LogP', 'HydrogenBondDonors',
                          'HydrogenBondAcceptors', 'TopologicalPolarSurfaceArea',
                          'NumberofRotatableBonds', 'NumberofValenceElectrons',
                          'NumberofAromaticRings', 'Fractionofsp3Carbons',
                          'Asphericity', 'Eccentricity', 'NPR1', 'NPR2',
                          'PMI1', 'PMI2', 'PMI3', 'RadiusofGyration',
                          'InertialShapeFactor', 'SpherocityIndex']

# Extract features and target
features = data[original_feature_names]
target = data['pIC50']

# Remove rows with NaN values
features = features.dropna()
target = target[features.index]

# Convert to array
X = features.values
y = target.values

# Normalize/Standardize features
scaler_X = StandardScaler()
X = scaler_X.fit_transform(X)

# Split data into 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Save the scaler for later use
joblib.dump(scaler_X, 'scaler_X_pIC50.pkl')

# Define and tune XGBoost model
xgb = XGBRegressor()

# Define the parameter grid for RandomizedSearchCV
param_distributions = {
    'n_estimators': randint(100, 1000),  # Start from a higher number to avoid very low values
    'learning_rate': uniform(0.01, 0.3),
    'max_depth': randint(4, 10),  # Slightly narrower range
    'subsample': uniform(0.7, 0.3),
    'colsample_bytree': uniform(0.7, 0.3),
    'reg_alpha': uniform(0.0, 1.0),  # L1 regularization
    'reg_lambda': uniform(0.0, 1.0)  # L2 regularization
}

# Implement RandomizedSearchCV without cross-validation
random_search_xgb = RandomizedSearchCV(
    xgb, param_distributions, 
    n_iter=200,  # Reduced number of iterations
    scoring='neg_mean_squared_error', 
    verbose=2,  # Increased verbosity for more detailed output
    n_jobs=-1, 
    random_state=42
)
random_search_xgb.fit(X_train, y_train)

# Best hyperparameters for XGBoost
best_params_xgb = random_search_xgb.best_params_
print(f"Best parameters for XGBoost: {best_params_xgb}")

# Train the final XGBoost model with the best parameters
best_xgb = XGBRegressor(**best_params_xgb)
best_xgb.fit(X_train, y_train)

# Save the trained XGBoost model
joblib.dump(best_xgb, 'xgboost_model_pIC50_optimized.pkl')

# Evaluate XGBoost model
y_train_pred_xgb = best_xgb.predict(X_train)
train_rmse_xgb = np.sqrt(mean_squared_error(y_train, y_train_pred_xgb))
train_r2_xgb = r2_score(y_train, y_train_pred_xgb)

y_test_pred_xgb = best_xgb.predict(X_test)
test_rmse_xgb = np.sqrt(mean_squared_error(y_test, y_test_pred_xgb))
test_r2_xgb = r2_score(y_test, y_test_pred_xgb)
print(y_train)
print(y_train_pred_xgb)
print(f"XGBoost - Train RMSE: {train_rmse_xgb:.6f}")
print(f"XGBoost - Train R²: {train_r2_xgb:.6f}")
print(f"XGBoost - Test RMSE: {test_rmse_xgb:.6f}")
print(f"XGBoost - Test R²: {test_r2_xgb:.6f}")


Fitting 5 folds for each of 200 candidates, totalling 1000 fits


KeyboardInterrupt: 

In [4]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem, Descriptors3D
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
import joblib

def compute_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        raise ValueError("Invalid SMILES string")

    # 2D descriptors
    descriptors = {
        'MolecularWeight': Descriptors.MolWt(mol),
        'LogP': Descriptors.MolLogP(mol),
        'HydrogenBondDonors': Descriptors.NumHDonors(mol),
        'HydrogenBondAcceptors': Descriptors.NumHAcceptors(mol),
        'TopologicalPolarSurfaceArea': Descriptors.TPSA(mol),
        'NumberofRotatableBonds': Descriptors.NumRotatableBonds(mol),
        'NumberofValenceElectrons': Descriptors.NumValenceElectrons(mol),
        'NumberofAromaticRings': Descriptors.NumAromaticRings(mol),
        'Fractionofsp3Carbons': Descriptors.FractionCSP3(mol)
    }

    # Generate 3D molecule and compute 3D descriptors
    mol_3d = generate_3d_molecule(mol)
    descriptors.update(compute_3d_descriptors(mol_3d))

    return descriptors

def generate_3d_molecule(mol):
    mol = Chem.AddHs(mol)
    best_mol = None
    best_energy = float('inf')

    for _ in range(100):
        mol_temp = Chem.Mol(mol)  # Ensure fresh copy each iteration
        AllChem.EmbedMolecule(mol_temp)
        AllChem.UFFOptimizeMolecule(mol_temp)
        energy = AllChem.UFFGetMoleculeForceField(mol_temp).CalcEnergy()
        
        if energy < best_energy:
            best_energy = energy
            best_mol = mol_temp
    
    return best_mol

def compute_3d_descriptors(mol):
    def safe_descriptor_calculation(func):
        try:
            return func(mol)
        except Exception:
            return np.nan

    return {
        'Asphericity': safe_descriptor_calculation(Descriptors3D.Asphericity),
        'Eccentricity': safe_descriptor_calculation(Descriptors3D.Eccentricity),
        'NPR1': safe_descriptor_calculation(Descriptors3D.NPR1),
        'NPR2': safe_descriptor_calculation(Descriptors3D.NPR2),
        'PMI1': safe_descriptor_calculation(Descriptors3D.PMI1),
        'PMI2': safe_descriptor_calculation(Descriptors3D.PMI2),
        'PMI3': safe_descriptor_calculation(Descriptors3D.PMI3),
        'RadiusofGyration': safe_descriptor_calculation(Descriptors3D.RadiusOfGyration),
        'InertialShapeFactor': safe_descriptor_calculation(Descriptors3D.InertialShapeFactor),
        'SpherocityIndex': safe_descriptor_calculation(Descriptors3D.SpherocityIndex)
    }

# Example SMILES input
smiles = 'OC1=CC=C(C=C1)C1=C(O)C(=O)C2=C(O)C=C(O)C=C2O1'

# Calculate the molecular descriptors
descriptors = compute_descriptors(smiles)
print("Descriptors:", descriptors)

# Convert descriptors into a DataFrame
original_feature_names = ['MolecularWeight', 'LogP', 'HydrogenBondDonors',
                          'HydrogenBondAcceptors', 'TopologicalPolarSurfaceArea',
                          'NumberofRotatableBonds', 'NumberofValenceElectrons',
                          'NumberofAromaticRings', 'Fractionofsp3Carbons',
                          'Asphericity', 'Eccentricity', 'NPR1', 'NPR2',
                          'PMI1', 'PMI2', 'PMI3', 'RadiusofGyration',
                          'InertialShapeFactor', 'SpherocityIndex']
descriptors_df = pd.DataFrame([descriptors], columns=original_feature_names)

# Load the saved scalers and model
scaler_X = joblib.load('scaler_X_pIC50.pkl')
xgb_model = joblib.load('xgboost_model_pIC50_optimized.pkl')

# Ensure descriptors match the model's expected feature names
descriptors_df = descriptors_df.reindex(columns=original_feature_names, fill_value=0)

# Scale the new descriptors
descriptors_scaled = scaler_X.transform(descriptors_df)

# Predict the scaled IC50
predicted_pIC50 = xgb_model.predict(descriptors_scaled)

predicted_pIC50_value = predicted_pIC50[0]
predicted_IC50_value = 10 ** (-predicted_pIC50_value)*1000000 
print(f"Predicted pIC50: {predicted_pIC50_value:.6f}")
print(f"Predicted IC50: {predicted_IC50_value:.6f}")

Descriptors: {'MolecularWeight': 286.23900000000003, 'LogP': 2.2824, 'HydrogenBondDonors': 4, 'HydrogenBondAcceptors': 6, 'TopologicalPolarSurfaceArea': 111.13000000000001, 'NumberofRotatableBonds': 1, 'NumberofValenceElectrons': 106, 'NumberofAromaticRings': 3, 'Fractionofsp3Carbons': 0.0, 'Asphericity': 0.5207156729252564, 'Eccentricity': 0.9800966743412197, 'NPR1': 0.19852080229860336, 'NPR2': 0.8211833158881978, 'PMI1': 760.0242627193776, 'PMI2': 3143.8480853841083, 'PMI3': 3828.4363851008097, 'RadiusofGyration': 3.6751510396144096, 'InertialShapeFactor': 0.0010804698694091585, 'SpherocityIndex': 0.058099608278420194}
Predicted pIC50: 4.662551
Predicted IC50: 21.749490


/home/aya/.local/lib/python3.7/site-packages/sklearn/base.py:444: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


[CV] END colsample_bytree=0.7692681476866446, learning_rate=0.0823076398078035, max_depth=7, n_estimators=554, reg_alpha=0.6099966577826209, reg_lambda=0.8331949117361643, subsample=0.7520093960523315; total time=   1.9s
[CV] END colsample_bytree=0.7836614057776545, learning_rate=0.2201073489918314, max_depth=6, n_estimators=503, reg_alpha=0.8870864242651173, reg_lambda=0.7798755458576239, subsample=0.8926094938462863; total time=   0.9s
[CV] END colsample_bytree=0.8772499781707032, learning_rate=0.01915007498171483, max_depth=7, n_estimators=781, reg_alpha=0.8226005606596583, reg_lambda=0.3601906414112629, subsample=0.7381181537955654; total time=   2.3s
[CV] END colsample_bytree=0.7320268197807652, learning_rate=0.11069801943908895, max_depth=5, n_estimators=447, reg_alpha=0.6468483616687238, reg_lambda=0.3882683839898722, subsample=0.7688184237114849; total time=   0.8s
[CV] END colsample_bytree=0.805176267641979, learning_rate=0.18697530605638993, max_depth=9, n_estimators=434, reg

[CV] END colsample_bytree=0.8173181822719722, learning_rate=0.0646708263364187, max_depth=7, n_estimators=101, reg_alpha=0.4251558744912447, reg_lambda=0.20794166286818883, subsample=0.8703100983459974; total time=   0.8s
[CV] END colsample_bytree=0.9422320465492187, learning_rate=0.278827389977048, max_depth=9, n_estimators=324, reg_alpha=0.2721322493846353, reg_lambda=0.6476901205413623, subsample=0.7001561130985947; total time=   1.1s
[CV] END colsample_bytree=0.7836614057776545, learning_rate=0.2201073489918314, max_depth=6, n_estimators=503, reg_alpha=0.8870864242651173, reg_lambda=0.7798755458576239, subsample=0.8926094938462863; total time=   0.8s
[CV] END colsample_bytree=0.8736594686522676, learning_rate=0.14154223690542608, max_depth=9, n_estimators=150, reg_alpha=0.46559801813246016, reg_lambda=0.5426446347075766, subsample=0.7859623756384853; total time=   0.5s
[CV] END colsample_bytree=0.8647679994118361, learning_rate=0.2243787768100187, max_depth=8, n_estimators=984, reg

[CV] END colsample_bytree=0.8123620356542087, learning_rate=0.2952142919229748, max_depth=6, n_estimators=171, reg_alpha=0.5986584841970366, reg_lambda=0.15601864044243652, subsample=0.7467983561008608; total time=   0.4s
[CV] END colsample_bytree=0.7609183674204307, learning_rate=0.2928560711673943, max_depth=6, n_estimators=342, reg_alpha=0.16122128725400442, reg_lambda=0.9296976523425731, subsample=0.942436113869325; total time=   0.4s
[CV] END colsample_bytree=0.890021126953127, learning_rate=0.2714381770563153, max_depth=7, n_estimators=719, reg_alpha=0.18657005888603584, reg_lambda=0.8925589984899778, subsample=0.8618026725746952; total time=   1.2s
[CV] END colsample_bytree=0.7252419894985146, learning_rate=0.05848861422838413, max_depth=9, n_estimators=203, reg_alpha=0.6064290596595899, reg_lambda=0.009197051616629648, subsample=0.7304414628598096; total time=   0.6s
[CV] END colsample_bytree=0.9790050504432495, learning_rate=0.031124839254863167, max_depth=5, n_estimators=387,

[CV] END colsample_bytree=0.8855158027999261, learning_rate=0.12473859738014881, max_depth=7, n_estimators=931, reg_alpha=0.4667628932479799, reg_lambda=0.8599404067363206, subsample=0.9040922615763338; total time=   2.2s
[CV] END colsample_bytree=0.9162819772756388, learning_rate=0.10241823755571676, max_depth=7, n_estimators=982, reg_alpha=0.6363326181858954, reg_lambda=0.25046181860558414, subsample=0.8769612542681631; total time=   2.1s
[CV] END colsample_bytree=0.72738600303584, learning_rate=0.10579409127712446, max_depth=9, n_estimators=834, reg_alpha=0.9506071469375561, reg_lambda=0.5734378881232861, subsample=0.8895511636509397; total time=   1.7s
[CV] END colsample_bytree=0.7384137516873317, learning_rate=0.0555708080536883, max_depth=9, n_estimators=619, reg_alpha=0.18006727234929853, reg_lambda=0.696501466227079, subsample=0.8234983643072584; total time=   2.0s
[CV] END colsample_bytree=0.9224382733417136, learning_rate=0.1819452547772166, max_depth=8, n_estimators=649, reg

[CV] END colsample_bytree=0.8173181822719722, learning_rate=0.0646708263364187, max_depth=7, n_estimators=101, reg_alpha=0.4251558744912447, reg_lambda=0.20794166286818883, subsample=0.8703100983459974; total time=   0.8s
[CV] END colsample_bytree=0.890021126953127, learning_rate=0.2714381770563153, max_depth=7, n_estimators=719, reg_alpha=0.18657005888603584, reg_lambda=0.8925589984899778, subsample=0.8618026725746952; total time=   1.2s
[CV] END colsample_bytree=0.7836614057776545, learning_rate=0.2201073489918314, max_depth=6, n_estimators=503, reg_alpha=0.8870864242651173, reg_lambda=0.7798755458576239, subsample=0.8926094938462863; total time=   0.8s
[CV] END colsample_bytree=0.8772499781707032, learning_rate=0.01915007498171483, max_depth=7, n_estimators=781, reg_alpha=0.8226005606596583, reg_lambda=0.3601906414112629, subsample=0.7381181537955654; total time=   2.4s
[CV] END colsample_bytree=0.8233620162616558, learning_rate=0.19083456460698955, max_depth=8, n_estimators=657, re

[CV] END colsample_bytree=0.8351497755908628, learning_rate=0.013979488347959958, max_depth=4, n_estimators=415, reg_alpha=0.5632882178455393, reg_lambda=0.3854165025399161, subsample=0.7047898756660642; total time=   1.2s
[CV] END colsample_bytree=0.9993221455146826, learning_rate=0.09003430428258549, max_depth=5, n_estimators=741, reg_alpha=0.4110370133182313, reg_lambda=0.033050732900548385, subsample=0.8035213744080049; total time=   1.7s
[CV] END colsample_bytree=0.8594063894704443, learning_rate=0.17219053648303195, max_depth=7, n_estimators=755, reg_alpha=0.9758520794625346, reg_lambda=0.5163003483011953, subsample=0.7968869418823737; total time=   1.4s
[CV] END colsample_bytree=0.72738600303584, learning_rate=0.10579409127712446, max_depth=9, n_estimators=834, reg_alpha=0.9506071469375561, reg_lambda=0.5734378881232861, subsample=0.8895511636509397; total time=   1.7s
[CV] END colsample_bytree=0.7384137516873317, learning_rate=0.0555708080536883, max_depth=9, n_estimators=619, 

[CV] END colsample_bytree=0.7692681476866446, learning_rate=0.0823076398078035, max_depth=7, n_estimators=554, reg_alpha=0.6099966577826209, reg_lambda=0.8331949117361643, subsample=0.7520093960523315; total time=   1.6s
[CV] END colsample_bytree=0.7523099287014974, learning_rate=0.21728132143073978, max_depth=5, n_estimators=971, reg_alpha=0.13752094414599325, reg_lambda=0.3410663510502585, subsample=0.7340420563721767; total time=   1.5s
[CV] END colsample_bytree=0.9507940361536618, learning_rate=0.2187922618281094, max_depth=8, n_estimators=740, reg_alpha=0.17329432007084578, reg_lambda=0.15643704267108605, subsample=0.7750728694493786; total time=   1.6s
[CV] END colsample_bytree=0.9222921582732004, learning_rate=0.2401584559325715, max_depth=4, n_estimators=100, reg_alpha=0.681039427096909, reg_lambda=0.23750647129246993, subsample=0.8200668678616074; total time=   0.2s
[CV] END colsample_bytree=0.9907907606857296, learning_rate=0.16692935325104466, max_depth=4, n_estimators=404, 

In [2]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem, Descriptors3D
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
import joblib

def compute_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        raise ValueError("Invalid SMILES string")

    # 2D descriptors
    descriptors = {
        'MolecularWeight': Descriptors.MolWt(mol),
        'LogP': Descriptors.MolLogP(mol),
        'HydrogenBondDonors': Descriptors.NumHDonors(mol),
        'HydrogenBondAcceptors': Descriptors.NumHAcceptors(mol),
        'TopologicalPolarSurfaceArea': Descriptors.TPSA(mol),
        'NumberofRotatableBonds': Descriptors.NumRotatableBonds(mol),
        'NumberofValenceElectrons': Descriptors.NumValenceElectrons(mol),
        'NumberofAromaticRings': Descriptors.NumAromaticRings(mol),
        'Fractionofsp3Carbons': Descriptors.FractionCSP3(mol)
    }

    # Generate 3D molecule and compute 3D descriptors
    mol_3d = generate_3d_molecule(mol)
    descriptors.update(compute_3d_descriptors(mol_3d))

    return descriptors

def generate_3d_molecule(mol):
    mol = Chem.AddHs(mol)
    best_mol = None
    best_energy = float('inf')

    for _ in range(100):
        mol_temp = Chem.Mol(mol)  # Ensure fresh copy each iteration
        AllChem.EmbedMolecule(mol_temp)
        AllChem.UFFOptimizeMolecule(mol_temp)
        energy = AllChem.UFFGetMoleculeForceField(mol_temp).CalcEnergy()
        
        if energy < best_energy:
            best_energy = energy
            best_mol = mol_temp
    
    return best_mol

def compute_3d_descriptors(mol):
    def safe_descriptor_calculation(func):
        try:
            return func(mol)
        except Exception:
            return np.nan

    return {
        'Asphericity': safe_descriptor_calculation(Descriptors3D.Asphericity),
        'Eccentricity': safe_descriptor_calculation(Descriptors3D.Eccentricity),
        'NPR1': safe_descriptor_calculation(Descriptors3D.NPR1),
        'NPR2': safe_descriptor_calculation(Descriptors3D.NPR2),
        'PMI1': safe_descriptor_calculation(Descriptors3D.PMI1),
        'PMI2': safe_descriptor_calculation(Descriptors3D.PMI2),
        'PMI3': safe_descriptor_calculation(Descriptors3D.PMI3),
        'RadiusofGyration': safe_descriptor_calculation(Descriptors3D.RadiusOfGyration),
        'InertialShapeFactor': safe_descriptor_calculation(Descriptors3D.InertialShapeFactor),
        'SpherocityIndex': safe_descriptor_calculation(Descriptors3D.SpherocityIndex)
    }

# Example SMILES input
smiles = 'OC1=CC=C(C=C1)C1=C(O)C(=O)C2=C(O)C=C(O)C=C2O1'

# Calculate the molecular descriptors
descriptors = compute_descriptors(smiles)
print("Descriptors:", descriptors)

# Convert descriptors into a DataFrame
original_feature_names = ['MolecularWeight', 'LogP', 'HydrogenBondDonors',
                          'HydrogenBondAcceptors', 'TopologicalPolarSurfaceArea',
                          'NumberofRotatableBonds', 'NumberofValenceElectrons',
                          'NumberofAromaticRings', 'Fractionofsp3Carbons',
                          'Asphericity', 'Eccentricity', 'NPR1', 'NPR2',
                          'PMI1', 'PMI2', 'PMI3', 'RadiusofGyration',
                          'InertialShapeFactor', 'SpherocityIndex']
descriptors_df = pd.DataFrame([descriptors], columns=original_feature_names)

# Load the saved scalers and model
scaler_X = joblib.load('scaler_X_pIC50.pkl')
gb_model = joblib.load('gradient_boosting_model_pIC50.pkl')

# Ensure descriptors match the model's expected feature names
descriptors_df = descriptors_df.reindex(columns=original_feature_names, fill_value=0)

# Scale the new descriptors
descriptors_scaled = scaler_X.transform(descriptors_df)

# Predict the scaled IC50
predicted_pIC50 = gb_model.predict(descriptors_scaled)

predicted_pIC50_value = predicted_pIC50[0]
predicted_IC50_value = 10 ** (-predicted_pIC50_value) * 1000000 
print(f"Predicted pIC50: {predicted_pIC50_value:.6f}")
print(f"Predicted IC50: {predicted_IC50_value:.6f}")


Descriptors: {'MolecularWeight': 286.23900000000003, 'LogP': 2.2824, 'HydrogenBondDonors': 4, 'HydrogenBondAcceptors': 6, 'TopologicalPolarSurfaceArea': 111.13000000000001, 'NumberofRotatableBonds': 1, 'NumberofValenceElectrons': 106, 'NumberofAromaticRings': 3, 'Fractionofsp3Carbons': 0.0, 'Asphericity': 0.520716768483806, 'Eccentricity': 0.9800967966184659, 'NPR1': 0.19852019861520756, 'NPR2': 0.821184046673551, 'PMI1': 760.0220053688383, 'PMI2': 3143.8511057479855, 'PMI3': 3828.4366561711527, 'RadiusofGyration': 3.675151285363365, 'InertialShapeFactor': 0.0010804740400576045, 'SpherocityIndex': 0.05809987329936968}


FileNotFoundError: [Errno 2] No such file or directory: 'gradient_boosting_model_pIC50.pkl'